# `Промышленное машинное обучение на Spark`
## `Задание 01: Веб-сервис исправления опечаток.`

<span style="color:red">Дедлайн: 5 февраля 00:00</span>

<span style="color:red">Заполненный ноутбук присылать на почту <b>ekolmagorov98@yandex.ru</b> с темой письма <b>[HSE Spark 2024][Задание 01][ФИО]</b>.</span>

В данном задании предлагается реализовать веб-сервис, написанный на фреймворке Flask по исправлению опечаток для русского языка. 

### Импорт требуемых модулей

In [2]:
import requests
import time

###  Задание 1. Разминка.

Допишите GET-метод `/version`, который при вызове будет возвращать номер версии вашего сервиса. Так как это первая версия, то в качестве ответа верните json-строку, в которой в поле `version` будет записана строка `1.0`.

*Замечание.* Не забывайте, чтобы формировать ответ в формате json [функцией jsonify](https://sky.pro/media/vozvrashhenie-json-otveta-iz-predstavleniya-flask/). 

In [3]:
%%writefile server.py

from flask import Flask
from flask import jsonify

app = Flask(__name__)
@app.route('/version', methods=['GET'])
def get_version():
    # YOUR CODE HERE
   
if __name__ == '__main__':
    app.run(host='localhost', port=5555)

Overwriting server.py


Магическая команда writefile производит запись содержимого из jupyter-ячейки в файл, который указывается после данной команды. Так как запуск и остановку веб-сервера необходимо производить в отдельном потоке исполнения, то дополнительно откройте окно терминала и через него произведите запуск сервера следующей командой:

```bash
> lsof -i:5555 | awk '{ if (NR>1) print $2}' | xargs -r kill -9 & python3 server.py
```

В случае успешного запуска, вы должны получить в терминале следующий вывод.

```
 * Serving Flask app 'server'
 * Debug mode: off
   WARNING: This is a development server. Do not use it in a production deployment. Use a   production WSGI server instead.
 * Running on http://http://127.0.0.1:5555
 * http://10.128.0.16:5555
Press CTRL+C to quit
```

Данный вовод, говорит о том, что произошел запуск приложения `server` на хосте `127.0.0.1` и порту `5555`. Чтобы остановить работу сервера необходимо нажать сочетание клавиш `CTRL+C`. 

<b><span style="color:red">!!! Важно: перезапускайте сервер после каждого обновления ячейки с кодом приложения, чтобы ваши изменения кода смогли попасть в него.</span></b>

Запустите приложение в соседнем окне терминала и проверьте правильность вашего решения по вызову кода ниже. Все ошибки на стороне сервера будут отражены в окне терминала.

In [5]:
resp = requests.get("http://localhost:5555/version")

assert resp.status_code == 200,  f'''Статус некорректный статус ответа: {resp.status_code}.
                                     Описание ошибки: {resp.reason}.
                                     Посмотрите в запущенном терминале более детальную информацию о ней'''
assert resp.json().get("version") == "1.0", "Некорректный номер версии"
resp.json()

{'version': '1.0'}

Теперь веб-приложение на метде `/version` выдаёт корректный статус ответа - код 200 - и то содержимое ответа, которое ожидалось.

### Задание 2. Поиск наиболее релевантного исправления.

Теперь реализуем целевое действие, которое требуется от приложения - исправление опечатки в слове.
В качестве используемого метода поиска опечатки воспользуемся алгоритмом поиска ближайшего слова по словарю, в качестве метрики близости между двумя словами воспользуемся [расстоянием Левенштейна](https://ru.wikipedia.org/wiki/Расстояние_Левенштейна). 
Данное расстояние между словами $word_{1}$ и $word_{2}$ определяется, как минимальное количество требуемых операций вставки, замены и удаления символов в слове $word_{1}$, чтобы его превратить в слово $word_{2}$.

Например:
- D(кот, кит) = 1, достаточно поменять букву 'о' на 'и';
- D(собака, чайка) = 4, необходимо удалить одну из букв + произвести замену оствшихся первых трёх на слог 'чай'

В качестве словар подготовлен txt-файл со всеми корректными словами и их количеством в романе Л.Н.Толстого "Война и мир".

В данном задании необходимо:
1. Cчитать данный файл в [структуру Counter](https://docs-python.ru/standart-library/modul-collections-python/klass-counter-modulja-collections/).
2. Реализовать функцию вычисления вероятности некоторого слова в словаре, как $P(word_{k}) = \frac{\#word_{k}}{\#word_{1}+...+\#word_{k}+...+\#word_{N}}$
3. Найти в словаре слово среди слов-кандидатов, которое имеет с максимальную вероятность - оно и будет исправленным ответом.
4. Подготовить GET-метод `/correct` с параметом `check_word`. Ответ дайте в формате json, в котором в поле `corrected` - будет располагаться исправленный вариант, если он будет найден в словаре. Чтобы передавать именнованные параметры в GET-методе необходимо воспользоваться модулем requests из flask. Обратите внимание на следующий [ответ со Stakoverflow](https://stackoverflow.com/a/24892131) по передаче параметров.

In [6]:
%%writefile server.py

from collections import Counter
from flask import Flask
from flask import jsonify
from flask import request

def read_dictionary(filename: str) -> Counter:
    '''
        Read dictionary file with words statistics.
        Function results is Counter datatype.
    '''
    #YOUR CODE HERE

WORDS = read_dictionary('dictionary.txt')


def P(word, N=sum(WORDS.values())): 
    '''
        Probability of `word`: (num occurances of `word`)/ (total count of words) 
    '''
    # YOUR CODE HERE

def most_probable(word): 
    '''
        Find most probable (with max ) spelling correction for word. 
        Hint: see max function + key param 
            https://www.programiz.com/python-programming/methods/built-in/max
    '''
    # YOUR CODE HERE

def candidates(word): 
    '''
        Generate most nearest spelling corrections for word.
        If found word in dictionary then return word, otherwise
        try found words from one and then two edit distance
    '''
    one_symbol_change = generate_candidates_one_symbol(word)
    two_symbol_change = generate_candidates_two_symbol(word)
    return (known([word]) or known(one_symbol_change) or known(two_symbol_change) or [word])

def known(words): 
    '''
        The subset of `words` that appear in the dictionary of WORDS.
    '''
    return set(w for w in words if w in WORDS)

def generate_candidates_one_symbol(word):
    '''
        Generate candidates that are one edit symbol away from `word`.
    '''
    
    letters    = 'абвгдеёжзиклмнопрстуфхцчшщъыьэюя'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def generate_candidates_two_symbol(word): 
    '''
        Generate all сandidates that are two edits away from `word`.
    '''
    return [
        e2 for e1 in generate_candidates_one_symbol(word)
        for e2 in generate_candidates_one_symbol(e1)
    ]

app = Flask(__name__)

@app.route('/correct', methods=['GET'])
def correct():
    # YOUR CODE HERE

if __name__ == '__main__':
    app.run(host='localhost', port=5555)

Overwriting server.py


Запустите приложение в окне терминала по команде описанной выше. После чего запустите ячейку ниже. Если всё выполненно корректно, то вы получите пустой ответ после её завершения.

In [7]:
params = {"check_word": "полкводиц"}
resp = requests.get("http://localhost:5555/correct", params=params)

assert resp.status_code == 200, "Код ошибки не равен 200, ошибки на стороне сервера"
assert resp.json()['correct_word'] == "полководец", "Исправление некорретное"

Проверим, как ведёт себя программа на более современных словах.

In [8]:
params = {"check_word": "радио"}
resp = requests.get("http://localhost:5555/correct", params=params)

print(resp.json())

{'correct_word': 'ради'}


Как можно заметить, приложение выдало некорректный ответ на слово "радио". Объясняется это тем, что словарь для проверки был сформирован на основании текста XIX века и привычное современному человеку слово "радио" там не встречается. Попробуем добавить в возможность добавления новых слов в словарь.

### Задание 3. Обогащение словаря новыми словами.

В данном задании предлагается реализовать возможность добавления новых слов в словарь, чтобы была возможность редактировать поведение программы на тех словах, которые там изначально не встречаются.

В данном задании предлагается к существующему решению добавить POST-метод `/add_word`, в котором необходимо считать поле `added_word` из тела запроса и добавить его в структуру Counter. 

- Подсказака 1. Можно посмотреть поля и их значения в запросе. Если взять атрибут json из входящего request. См. следующий [ответ](https://stackoverflow.com/a/35614301)
- Подсказка 2. Добавление/измнение значений в структуре Counter может быть выполнена следующим образом:
```python
counter = Counter({'Dog': 2, 'Cat': 1})
counter['Owl'] += 1
```
- Подсказка 3. Так как в методе добавления достаточно проинформировать клиента об успешном добавлении, то для этого достаточно просто отослать пустой ответ со статусом 200: `jsonify(success=True)`

In [13]:

%%writefile server.py

from collections import Counter
from flask import Flask
from flask import jsonify, request, abort

def read_dictionary(filename: str) -> Counter:
    '''
        Read dictionary file with words statistics.
        Function results is Counter datatype.
    '''
    #YOUR CODE HERE FROM PREVIOUS CELL

WORDS = read_dictionary('dictionary.txt')


def P(word, N=sum(WORDS.values())): 
    '''
        Probability of `word`: (num occurances of `word`)/ (total count of words) 
    '''
    # YOUR CODE HERE FROM PREVIOUS CELL

def most_probable(word): 
    '''
        Find most probable (with max ) spelling correction for word. 
        Hint: see max function + key param 
            https://www.programiz.com/python-programming/methods/built-in/max
    '''
    # YOUR CODE HERE FROM PREVIOUS CELL

def candidates(word): 
    '''
        Generate most nearest spelling corrections for word.
        If found word in dictionary then return word, otherwise
        try found words from one and then two edit distance
    '''
    one_symbol_change = generate_candidates_one_symbol(word)
    two_symbol_change = generate_candidates_two_symbol(word)
    return (known([word]) or known(one_symbol_change) or known(two_symbol_change) or [word])

def known(words): 
    '''
        The subset of `words` that appear in the dictionary of WORDS.
    '''
    return set(w for w in words if w in WORDS)

def generate_candidates_one_symbol(word):
    '''
        Generate candidates that are one edit symbol away from `word`.
    '''
    
    letters    = 'абвгдеёжзиклмнопрстуфхцчшщъыьэюя'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def generate_candidates_two_symbol(word): 
    '''
        Generate all сandidates that are two edits away from `word`.
    '''
    return [
        e2 for e1 in generate_candidates_one_symbol(word)
        for e2 in generate_candidates_one_symbol(e1)
    ]

app = Flask(__name__)

@app.route('/correct', methods=['GET'])
def correct():
    # YOUR CODE HERE FROM PREVIOUS CELL

@app.route('/add_word', methods=['POST'])
def add_word():
    # YOUR CODE HERE
    # ... 

    

if __name__ == '__main__':
    app.run(host='localhost', port=5555)

Overwriting server.py


Проверка нового метода

In [14]:
# Проверим работоспособность метода добавления нового слова
data = {"added_word": "радио"}
resp = requests.post("http://localhost:5555/add_word", json=data)

assert resp.status_code == 200, "Код ошибки не равен 200, ошибки на стороне сервера"

# Проверим, что новое слово действительно добавилось в словарь

params = {"check_word": "радио"}
resp = requests.get("http://localhost:5555/correct", params=params)


assert resp.status_code == 200, "Код ошибки не равен 200, ошибки на стороне сервера"
assert resp.json()['correct_word'] == "радио", "Новое слово радио не было добавлено в словарь корректных слов"


print(resp.json())

{'correct_word': 'радио'}


### Задание 4. Показываем слова кандидаты.

Добавьте параметризированный GET-метод `/candidates/<int:edit_distance>`, который будет выдавать в поле `words` своего ответа слова-кандидаты для слова `word`, передаваемого в параметрах запроса. Последнее значение в пути метода указывает, на каком расстоянии редактирования должны быть слова-кандидаты на исправление: здесь допустимо два значения 1 и 2.

Например, вызов метода `/candidates/1?word=кот` должен выдать следующие слова: "тот", "скот", "вот", "пот" и тд. 

А вызов `/candidates/2?word=кот`, следующие слова: "лож", "год", "вор", "пост", "коса" и тд.

In [15]:
%%writefile server.py

from collections import Counter
from flask import Flask
from flask import jsonify, request, abort

def read_dictionary(filename: str) -> Counter:
    '''
        Read dictionary file with words statistics.
        Function results is Counter datatype.
    '''
    #YOUR CODE HERE FROM PREVIOUS CELL

WORDS = read_dictionary('dictionary.txt')

def known(words): 
    '''
        The subset of `words` that appear in the dictionary of WORDS.
    '''
    return set(w for w in words if w in WORDS)

def generate_candidates_one_symbol(word):
    '''
        Generate candidates that are one edit symbol away from `word`.
    '''
    
    letters    = 'абвгдеёжзиклмнопрстуфхцчшщъыьэюя'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def generate_candidates_two_symbol(word): 
    '''
        Generate all сandidates that are two edits away from `word`.
    '''
    return [
        e2 for e1 in generate_candidates_one_symbol(word)
        for e2 in generate_candidates_one_symbol(e1)
    ]

app = Flask(__name__)

@app.route('/candidates/<int:edit_distance>', methods=['GET'])
def candidates(edit_distance: int):
    # YOUR CODE HERE
    

    

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5555)

Overwriting server.py


Проверка корректности добавленных методов

In [16]:

resp = requests.get("http://localhost:5555/candidates/1", params={"word": "генерал"})
assert resp.status_code == 200, "Код ответа не равен 200, в решении присутствуют ошибки"
print("Ответ от сервера:", resp.json())
assert len(resp.json()["words"]) == 5, f'''Ошибка в логике работы, для данного
                слова должно быть сформировано 5 кандидатов, получено {len(resp.json()['words'])}'''

resp = requests.get("http://localhost:5555/candidates/2", params={"word": "генерал"})
assert resp.status_code == 200, "Код ответа не равен 200, в решении присутствуют ошибки"
print("Ответ от сервера:", resp.json())
assert len(resp.json()["words"]) == 9, f'''Ошибка в логике работы, для данного слова должно 
                быть сформировано 9 кандидатов, получено {len(resp.json()['words'])}'''

Ответ от сервера: {'words': ['генерала', 'генералу', 'генерале', 'генералы', 'генерал']}
Ответ от сервера: {'words': ['генерала', 'генералу', 'генералах', 'генералом', 'генерале', 'генералов', 'генералам', 'генералы', 'генерал']}


#### Задание 5.

Как в примере из второй части первой лекции произведите упаковку приложения в Docker-контейнер, и произведите запуск этого контейнера.
Заполните пропущенные строки внутри Dockerfile. Произведите сборку образа и старт контейнера, запустив ячейки ниже.

In [17]:
%%writefile Dockerfile

# Используем в качестве базового образа образ python
FROM python:3.11-slim

# Создайте папку, в которой будет храниться исходные файлы приложения, с именем app
# YOUR CODE HERE

# Пометьте созданную папку app как рабочую директорию. 
# YOUR CODE HERE

# Перекопируйте файл server.py с текущей директории в созданную выше папку app
# YOUR CODE HERE

# Перекопируйте файл словаря c текущей директории в созданную выше папку app
# YOUR CODE HERE

# Установим библиотеку Flask внутрь контейнера
RUN pip3 install -q Flask

# Сделаем порт 5555, на котором работет приложение видимым
EXPOSE 5555

# Установим команду, которая будет запускаться при старте котнейнера
CMD ["python3", "server.py"]


Overwriting Dockerfile


Произведём сборку образа в качестве названия будет имя `corrector:1.0`. 

<b>Замечание:</b> не забывайте пересобирать образ после каждого обновления кода приложения.

In [25]:
# Удаление старого образа
! docker rmi corrector:1.0
# Создание нового
! docker build --progress=plain --no-cache --rm=true  -t corrector:1.0 .

Untagged: corrector:1.0
Deleted: sha256:64331db1efd6efdcf20e2f1af3f4f08d7f975789d3856283255772e3b51ce672


Посмотрим на существующие в системе Docker-образы и убедимся, что образ приложения существует в системе c нужным именем и тегом.

In [21]:
! docker images

REPOSITORY   TAG       IMAGE ID       CREATED             SIZE
corrector    1.0       177b957a0a0a   2 seconds ago       148MB
<none>       <none>    58d703a39ec4   About an hour ago   148MB
test_app     1.0       136ed896d9ec   5 days ago          131MB


Теперь запустим контейнер с приложением и проверим его работоспособность.

In [22]:
# Останавливаем старый конейнер с приложением 
! if [ "$(docker ps | grep -c 'corrector_app')" -gt 0 ]; then docker stop corrector_app; fi

# Запускаем новый контейнер
! docker run --name corrector_app --rm -d -p 8000:5555  corrector:1.0;

# Прежде чем слать запросы на веб-сервер подождём 2 секунды, пока он запустится
time.sleep(2.0)

try:
    resp = requests.get("http://localhost:8000/candidates/1", params={"word": "генерал"})
except Exception as ex:
    print("\033[91m В решении ошибка присутствует ошибка.", ex, "\033[0m")

assert resp.status_code == 200, "Код ответа не равен 200, в решении присутствуют ошибки"

if resp.status_code == 200:
    print("\033[92mВаш контейнеризированное приложение корректно работает!\033[0m")

corrector_app
a2ed8cc0e4517c5f4bc19d97e47819c12dfe24aaf71af0057a1bcfa9bebc0b3f
Ваш контейнеризированное приложение корректно работает!


Если ячейка выше выдаёт ошибку, то нужно смотреть причину в логах контейнера, Чтобы посмотреть логи и ошибки контейнеризированного приложения, воспользуйтесь следующей командой. 

In [23]:
! docker logs corrector_app

 * Serving Flask app 'server'
 * Debug mode: off
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5555
 * Running on http://172.17.0.2:5555
Press CTRL+C to quit
172.17.0.1 - - [18/Jan/2024 22:35:22] "GET /candidates/1?word=генерал HTTP/1.1" 200 -
